# 에이전트 평가(Evaluating Agents)

이메일 어시스턴트는 라우터를 사용하여 이메일을 분류한 후 응답 생성을 위해 에이전트에게 전달합니다. 실제 운영 환경에서 제대로 작동할지 어떻게 확신할 수 있을까요? 그래서 테스트가 중요한 이유입니다: 응답 품질, 토큰 사용량, 지연 시간, 분류 정확도 같은 정량적 지표를 통해 에이전트 아키텍처에 대한 결정을 안내합니다. 

[LangSmith](https://docs.smith.langchain.com/)는 에이전트 테스트를 위한 두 가지 주요 방법을 제공합니다. 

![overview-img](assets/overview_eval.png)

## Resources

- Notebook Reference: [evaluation.ipynb](https://github.com/langchain-ai/agents-from-scratch/blob/main/notebooks/evaluation.ipynb)
- For running evaluations: [test_tools.py](https://github.com/langchain-ai/agents-from-scratch/blob/main/notebooks/test_tools.py), [/tests/](https://github.com/langchain-ai/agents-from-scratch/tree/main/tests)
- For LangSmith Studio: [src/email_assistant](https://github.com/langchain-ai/agents-from-scratch/tree/main/src/email_assistant)
- Slides: [Building Ambient Agents with LangGraph - Building Agents & Evaluations.pdf](https://files.cdn.thinkific.com/file_uploads/967498/attachments/5f6/a6b/958/Building_Ambient_Agents_with_LangGraph_-_Building_Agents___Evaluations.pdf)